# Env Setup

In [3]:
pip install langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 5.8 MB/s eta 0:00:00


In [5]:
import os
os.environ["LANGSMITH_TRACING"]="true"
os.environ["LANGSMITH_API_KEY"] =""

In [6]:
import os
from langchain.chat_models import init_chat_model
os.environ["OPENAI_API_KEY"] = ""
model = init_chat_model("gpt-4.1")


# Langchain Docs



## Define Skills

In [ ]:
from typing import TypedDict

class Skill(TypedDict):
    name: str
    description: str
    content: str

SKILLS: list[Skill] = [
    {
    "name": "fairouz",
    "description":
        "Use this skill whenever the user asks about Fairouz or her songs. "
        "It provides structured cultural and musical explanations."
    ,
    "content": """
          You are a cultural music expert specializing in Fairouz and her work.

          For every question about a Fairouz song, respond in the following structured format:

          Lyrics Overview:
          - Briefly describe the theme, emotions, and key imagery of the song’s lyrics.
          - Do not quote full lyrics; summarize and interpret instead.

          Author & Composer:
          - Identify the lyricist and composer (e.g., the Rahbani Brothers, Ziad Rahbani).
          - If the authorship is uncertain or disputed, clearly state that.

          Origin & Inspiration:
          - Explain the cultural, historical, or personal inspiration behind the song.
          - Mention whether it reflects Lebanese folklore, political context, nostalgia, love, exile, nature, or national identity.

          Cultural Significance (optional but encouraged):
          - Describe why the song is important or beloved in Lebanese or Arab culture.

          OUTPUT FORMAT (use exactly):
          Lyrics Overview:
          ...

          Author & Composer:
          ...

          Origin & Inspiration:
          ...

          Cultural Significance:
          ...
          """,
},
]


## Skill Loading Tool

In [ ]:
from langchain.tools import tool

@tool
def load_skill(skill_name: str) -> str:
    """Load the full content of a skill into the agent's context.

    Use this when you need detailed information about how to handle a specific
    type of request. This will provide you with comprehensive instructions,
    policies, and guidelines for the skill area.

    Args:
        skill_name: The name of the skill to load (e.g., "expense_reporting", "travel_booking")
    """
    # Find and return the requested skill
    for skill in SKILLS:
        if skill["name"] == skill_name:
            return f"Loaded skill: {skill_name}\n\n{skill['content']}"

    # Skill not found
    available = ", ".join(s["name"] for s in SKILLS)
    return f"Skill '{skill_name}' not found. Available skills: {available}"

## Skill Middleware

In [ ]:
from langchain.agents.middleware import ModelRequest, ModelResponse, AgentMiddleware
from langchain.messages import SystemMessage
from typing import Callable

class SkillMiddleware(AgentMiddleware):
    """Middleware that injects skill descriptions into the system prompt."""

    # Register the load_skill tool as a class variable
    tools = [load_skill]

    def __init__(self):
        """Initialize and generate the skills prompt from SKILLS."""
        # Build skills prompt from the SKILLS list
        skills_list = []
        for skill in SKILLS:
            skills_list.append(
                f"- **{skill['name']}**: {skill['description']}"
            )
        self.skills_prompt = "\n".join(skills_list)

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        """Sync: Inject skill descriptions into system prompt."""
        # Build the skills addendum
        skills_addendum = (
            f"\n\n## Available Skills\n\n{self.skills_prompt}\n\n"
            "Use the load_skill tool when you need detailed information "
            "about handling a specific type of request."
        )

        # Append to system message content blocks
        new_content = list(request.system_message.content_blocks) + [
            {"type": "text", "text": skills_addendum}
        ]
        print(self.skills_prompt)
        new_system_message = SystemMessage(content=new_content)
        modified_request = request.override(system_message=new_system_message)
        return handler(modified_request)

## Agent with Skill Support

In [ ]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

# Create the agent with skill support
agent = create_agent(
    model,
    system_prompt=(
        "You are a helpful assistant. "
        "Your user might ask you questions about Fairouz. "
        "When they do, you MUST use the load_skill tool to load the 'fairouz' skill first, "
        "then follow its instructions to answer the question."
    ),
    middleware=[SkillMiddleware()],
    checkpointer=InMemorySaver(),
)

## Test Agent

In [ ]:
import uuid

# Configuration for this conversation thread
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

# Ask for a SQL query
result = agent.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "badaak ala bali"
                ),
            }
        ]
    },
    config
)

# Print the conversation
for message in result["messages"]:
    if hasattr(message, 'pretty_print'):
        message.pretty_print()
    else:
        print(f"{message.type}: {message.content}")

- **fairouz**: Use this skill whenever the user asks about Fairouz or her songs. It provides structured cultural and musical explanations.
- **fairouz**: Use this skill whenever the user asks about Fairouz or her songs. It provides structured cultural and musical explanations.
================================ Human Message =================================

badaak ala bali
================================== Ai Message ==================================
Tool Calls:
  load_skill (call_mFjlvA4uArcPeitJTeTvPHND)
 Call ID: call_mFjlvA4uArcPeitJTeTvPHND
  Args:
    skill_name: fairouz
================================= Tool Message =================================
Name: load_skill

Loaded skill: fairouz


          You are a cultural music expert specializing in Fairouz and her work.

          For every question about a Fairouz song, respond in the following structured format:

          Lyrics Overview:
          - Briefly describe the theme, emotions, and key imagery of the song’s lyrics.

In [ ]:
agent = create_agent(
    model,
    tools=[load_skill],  # Tool is available
    system_prompt=(
        "You are a helpful assistant. "
        "When you need detailed instructions for handling specific topics, "
        "use the available tools to load the relevant skill content."
    ),
    middleware=[SkillMiddleware()],
    checkpointer=InMemorySaver(),
)

# Langchain Example

In [ ]:
import uuid
from typing import TypedDict, NotRequired
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import ModelRequest, ModelResponse, AgentMiddleware
from langchain.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from typing import Callable

# Define skill structure
class Skill(TypedDict):
    """A skill that can be progressively disclosed to the agent."""
    name: str
    description: str
    content: str

# Define skills with schemas and business logic
SKILLS: list[Skill] = [
    {
        "name": "sales_analytics",
        "description": "Database schema and business logic for sales data analysis including customers, orders, and revenue.",
        "content": """# Sales Analytics Schema

## Tables

### customers
- customer_id (PRIMARY KEY)
- name
- email
- signup_date
- status (active/inactive)
- customer_tier (bronze/silver/gold/platinum)

### orders
- order_id (PRIMARY KEY)
- customer_id (FOREIGN KEY -> customers)
- order_date
- status (pending/completed/cancelled/refunded)
- total_amount
- sales_region (north/south/east/west)

### order_items
- item_id (PRIMARY KEY)
- order_id (FOREIGN KEY -> orders)
- product_id
- quantity
- unit_price
- discount_percent

## Business Logic

**Active customers**: status = 'active' AND signup_date <= CURRENT_DATE - INTERVAL '90 days'

**Revenue calculation**: Only count orders with status = 'completed'. Use total_amount from orders table, which already accounts for discounts.

**Customer lifetime value (CLV)**: Sum of all completed order amounts for a customer.

**High-value orders**: Orders with total_amount > 1000

## Example Query

-- Get top 10 customers by revenue in the last quarter
SELECT
    c.customer_id,
    c.name,
    c.customer_tier,
    SUM(o.total_amount) as total_revenue
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.status = 'completed'
  AND o.order_date >= CURRENT_DATE - INTERVAL '3 months'
GROUP BY c.customer_id, c.name, c.customer_tier
ORDER BY total_revenue DESC
LIMIT 10;
""",
    },
    {
        "name": "inventory_management",
        "description": "Database schema and business logic for inventory tracking including products, warehouses, and stock levels.",
        "content": """# Inventory Management Schema

## Tables

### products
- product_id (PRIMARY KEY)
- product_name
- sku
- category
- unit_cost
- reorder_point (minimum stock level before reordering)
- discontinued (boolean)

### warehouses
- warehouse_id (PRIMARY KEY)
- warehouse_name
- location
- capacity

### inventory
- inventory_id (PRIMARY KEY)
- product_id (FOREIGN KEY -> products)
- warehouse_id (FOREIGN KEY -> warehouses)
- quantity_on_hand
- last_updated

### stock_movements
- movement_id (PRIMARY KEY)
- product_id (FOREIGN KEY -> products)
- warehouse_id (FOREIGN KEY -> warehouses)
- movement_type (inbound/outbound/transfer/adjustment)
- quantity (positive for inbound, negative for outbound)
- movement_date
- reference_number

## Business Logic

**Available stock**: quantity_on_hand from inventory table where quantity_on_hand > 0

**Products needing reorder**: Products where total quantity_on_hand across all warehouses is less than or equal to the product's reorder_point

**Active products only**: Exclude products where discontinued = true unless specifically analyzing discontinued items

**Stock valuation**: quantity_on_hand * unit_cost for each product

## Example Query

-- Find products below reorder point across all warehouses
SELECT
    p.product_id,
    p.product_name,
    p.reorder_point,
    SUM(i.quantity_on_hand) as total_stock,
    p.unit_cost,
    (p.reorder_point - SUM(i.quantity_on_hand)) as units_to_reorder
FROM products p
JOIN inventory i ON p.product_id = i.product_id
WHERE p.discontinued = false
GROUP BY p.product_id, p.product_name, p.reorder_point, p.unit_cost
HAVING SUM(i.quantity_on_hand) <= p.reorder_point
ORDER BY units_to_reorder DESC;
""",
    },
]

# Create skill loading tool
@tool
def load_skill(skill_name: str) -> str:
    """Load the full content of a skill into the agent's context.

    Use this when you need detailed information about how to handle a specific
    type of request. This will provide you with comprehensive instructions,
    policies, and guidelines for the skill area.

    Args:
        skill_name: The name of the skill to load (e.g., "sales_analytics", "inventory_management")
    """
    # Find and return the requested skill
    for skill in SKILLS:
        if skill["name"] == skill_name:
            return f"Loaded skill: {skill_name}\n\n{skill['content']}"

    # Skill not found
    available = ", ".join(s["name"] for s in SKILLS)
    return f"Skill '{skill_name}' not found. Available skills: {available}"

# Create skill middleware
class SkillMiddleware(AgentMiddleware):
    """Middleware that injects skill descriptions into the system prompt."""

    # Register the load_skill tool as a class variable
    tools = [load_skill]

    def __init__(self):
        """Initialize and generate the skills prompt from SKILLS."""
        # Build skills prompt from the SKILLS list
        skills_list = []
        for skill in SKILLS:
            skills_list.append(
                f"- **{skill['name']}**: {skill['description']}"
            )
        self.skills_prompt = "\n".join(skills_list)

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        """Sync: Inject skill descriptions into system prompt."""
        # Build the skills addendum
        skills_addendum = (
            f"\n\n## Available Skills\n\n{self.skills_prompt}\n\n"
            "Use the load_skill tool when you need detailed information "
            "about handling a specific type of request."
        )

        # Append to system message content blocks
        new_content = list(request.system_message.content_blocks) + [
            {"type": "text", "text": skills_addendum}
        ]
        new_system_message = SystemMessage(content=new_content)
        modified_request = request.override(system_message=new_system_message)
        return handler(modified_request)

# Initialize your chat model (replace with your model)
# Example: from langchain_anthropic import ChatAnthropic
# model = ChatAnthropic(model="claude-3-5-sonnet-20241022")
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4")

# Create the agent with skill support
agent = create_agent(
    model,
    system_prompt=(
        "You are a SQL query assistant that helps users "
        "write queries against business databases."
    ),
    middleware=[SkillMiddleware()],
    checkpointer=InMemorySaver(),
)

# Example usage
if __name__ == "__main__":
    # Configuration for this conversation thread
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}

    # Ask for a SQL query
    result = agent.invoke(
        {
            "messages": [
                {
                    "role": "user",
                    "content": (
                        "Write a SQL query to find all customers "
                        "who made orders over $1000 in the last month"
                    ),
                }
            ]
        },
        config
    )

    # Print the conversation
    for message in result["messages"]:
        if hasattr(message, 'pretty_print'):
            message.pretty_print()
        else:
            print(f"{message.type}: {message.content}")

================================ Human Message =================================

Write a SQL query to find all customers who made orders over $1000 in the last month
================================== Ai Message ==================================
Tool Calls:
  load_skill (call_ySVaHRbqVXDPeH3Adsq7QaDX)
 Call ID: call_ySVaHRbqVXDPeH3Adsq7QaDX
  Args:
    skill_name: sales_analytics
================================= Tool Message =================================
Name: load_skill

Loaded skill: sales_analytics

# Sales Analytics Schema

## Tables

### customers
- customer_id (PRIMARY KEY)
- name
- email
- signup_date
- status (active/inactive)
- customer_tier (bronze/silver/gold/platinum)

### orders
- order_id (PRIMARY KEY)
- customer_id (FOREIGN KEY -> customers)
- order_date
- status (pending/completed/cancelled/refunded)
- total_amount
- sales_region (north/south/east/west)

### order_items
- item_id (PRIMARY KEY)
- order_id (FOREIGN KEY -> orders)
- product_id
- quantity
- unit_pric

# Medical Context - Agentic Skills Example

In [2]:
import uuid
from typing import TypedDict
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.agents.middleware import ModelRequest, ModelResponse, AgentMiddleware
from langchain.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver
from typing import Callable


In [24]:
# ─────────────────────────────────────────────
# Skill definition
# ─────────────────────────────────────────────

class Skill(TypedDict):
    name: str
    description: str
    content: str


SKILLS: list[Skill] = [
    {
        "name": "search_decision_skill",
        "description": "Decide whether a user query requires literature search.",
        "content": """# Search Decision Skill

Return ONE decision:

##DECISION##:
- SEARCH_REQUIRED

OR

##DECISION##:
- ANSWER_FROM_CONTEXT

OR

##DECISION##:
- ASK_CLARIFICATION: <short question>


NOTE: - Stop immediately after the decision and don't output any more text
"""
    },
    {
        "name": "clinical_answer_evaluator",
        "description": "Evaluate answer quality and safety.",
        "content": "... (your existing one) ..."
    },
    {
        "name": "citation_verifier_skill",
        "description": "Verify citations support claims.",
        "content": """# Citation Verification Skill
Return:

##DECISION##:
- CITATIONS_VALID

OR

##DECISION##:
- CITATION_MISMATCH: <reason>
"""
    },
]


In [25]:
@tool
def load_skill(skill_name: str) -> str:
    """Load the full content of a skill into the agent's context."""
    for skill in SKILLS:
        if skill["name"] == skill_name:
            return f"Loaded skill: {skill_name}\n\n{skill['content']}"
    return f"Skill '{skill_name}' not found."



class SkillMiddleware(AgentMiddleware):
    tools = [load_skill]

    def __init__(self):
        skills_list = [
            f"- **{skill['name']}**: {skill['description']}"
            for skill in SKILLS
        ]
        self.skills_prompt = "\n".join(skills_list)

    def wrap_model_call(
        self,
        request: ModelRequest,
        handler: Callable[[ModelRequest], ModelResponse],
    ) -> ModelResponse:
        skills_addendum = (
            f"\n\n## Available Skills\n\n{self.skills_prompt}\n\n"
            "Use the load_skill tool when you need to evaluate a clinical response."
        )

        new_content = list(request.system_message.content_blocks) + [
            {"type": "text", "text": skills_addendum}
        ]
        new_system_message = SystemMessage(content=new_content)
        return handler(request.override(system_message=new_system_message))

In [26]:


from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4.1")

agent = create_agent(
    model,
    system_prompt=(
        "You are a medical assistant chatbot that supports special skills.\n"
        "When a user query requires a specialized capability (like deciding whether "
        "to search literature, evaluating an answer, or verifying citations), "
        "you should explicitly load the appropriate skill using the load_skill tool.\n\n"
        "Process for every user query:\n"
        "1. Analyze the user question.\n"
        "2. Decide if a skill should be loaded (e.g., search_decision_skill, "
        "clinical_answer_evaluator, citation_verifier_skill).\n"
        "3. If a skill is needed, call the load_skill tool with the skill name.\n"
        "4. Use the loaded skill content and follow its instructions.\n\n"
        "Do not answer anything outside of the guildelines from the skill"
    ),
    middleware=[SkillMiddleware()],
    checkpointer=InMemorySaver(),
)


In [27]:
if __name__ == "__main__":
    thread_id = str(uuid.uuid4())
    config = {"configurable": {"thread_id": thread_id}}

    result = agent.invoke(
        {
            "messages": [
                {
                    "role": "user",
                    "content": (
                        "User question:\n"
                        "how to treat lower back pain?\n\n"
                    ),
                }
            ]
        },
        config
    )

    for message in result["messages"]:
        print(message.content)

User question:
how to treat lower back pain?



Loaded skill: search_decision_skill

# Search Decision Skill

Return ONE decision:

##DECISION##:
- SEARCH_REQUIRED

OR

##DECISION##:
- ANSWER_FROM_CONTEXT

OR

##DECISION##:
- ASK_CLARIFICATION: <short question>


NOTE: - Stop immediately after the decision and don't output any more text

##DECISION##:
- ANSWER_FROM_CONTEXT


In [28]:
# One conversation thread
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

print("\n🧠 Meddit Skill Demo Chatbot")
print("Type 'exit' to quit.\n")

while True:
    user_input = input("👤 User: ").strip()
    if user_input.lower() in {"exit", "quit"}:
        print("\n👋 Goodbye!")
        break

    result = agent.invoke(
        {
            "messages": [
                {"role": "user", "content": user_input}
            ]
        },
        config
    )

    # Print assistant reply
    for msg in reversed(result["messages"]):
        if msg.type == "ai":
            print(f"\n🤖 Assistant: {msg.content}\n")
            break


🧠 Meddit Skill Demo Chatbot
Type 'exit' to quit.

👤 User: how to treat tmj

🤖 Assistant: ##DECISION##:
- SEARCH_REQUIRED

👤 User: what are the new research regarding covid 19 vaccines

🤖 Assistant: ##DECISION##:
- SEARCH_REQUIRED

👤 User: how are you today

🤖 Assistant: I'm here to assist you with medical questions or concerns. How can I help you today?

👤 User: how do you evaluate the following: Upper gastrointestinal (GI) bleeding in adults is most commonly caused by peptic ulcer disease, gastritis, esophagitis, variceal bleeding, Mallory-Weiss syndrome, and malignancy, and requires prompt evaluation and management to reduce morbidity and mortality.

🤖 Assistant: Loaded the clinical_answer_evaluator skill. Please provide the answer you want evaluated, or let me know if you want an assessment of the medical statement provided.



KeyboardInterrupt: Interrupted by user